<a href="https://colab.research.google.com/github/elichen/aoc2023/blob/main/Day_24_Never_Tell_Me_The_Odds.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
input = """19, 13, 30 @ -2,  1, -2
18, 19, 22 @ -1, -1, -2
20, 25, 34 @ -2, -2, -4
12, 31, 28 @ -1, -2, -1
20, 19, 15 @  1, -5, -3""".split('\n')

In [ ]:
input = [x.rstrip() for x in open("input.txt").readlines()]

In [ ]:
# Function to parse each string and convert it into a tuple of tuples
def parse_string(s):
    parts = s.split('@')
    coords = tuple(map(int, parts[0].split(',')))
    deltas = tuple(map(int, parts[1].split(',')))
    return (coords, deltas)

stones = [parse_string(s) for s in input]

In [ ]:
def will_paths_intersect(stone1, stone2, min_range, max_range):
    # Extract coordinates and deltas from the stones
    (x1, y1, _), (dx1, dy1, _) = stone1
    (x2, y2, _), (dx2, dy2, _) = stone2

    # Avoid parallel movement scenario
    if dx1 == dx2 and dy1 == dy2:
        return None

    # Solving the system of linear equations for t1 and t2
    try:
        # Solving for t1 and t2 separately
        # x1 + dx1*t1 = x2 + dx2*t2 => t1 = (x2 - x1 + dx2*t2) / dx1
        # y1 + dy1*t1 = y2 + dy2*t2 => t1 = (y2 - y1 + dy2*t2) / dy1

        # Only proceed if the denominators are non-zero
        if dx1 != 0 and dy1 != 0:
            # Combine equations to eliminate t1
            # (x2 - x1 + dx2*t2) / dx1 = (y2 - y1 + dy2*t2) / dy1
            # Solving for t2
            t2 = (dx1 * (y2 - y1) - dy1 * (x2 - x1)) / (dy1 * dx2 - dx1 * dy2)

            # Calculate corresponding t1
            t1 = (x2 - x1 + dx2 * t2) / dx1

            # Calculate intersection point
            intersect_x = x1 + dx1 * t1
            intersect_y = y1 + dy1 * t1

            # Check if intersection is within the specified range and time is positive
            if min_range <= intersect_x <= max_range and min_range <= intersect_y <= max_range and t1 >= 0 and t2 >= 0:
                return intersect_x, intersect_y
    except ZeroDivisionError:
        # Division by zero indicates parallel movement, no intersection
        pass

    return None

print(will_paths_intersect(stones[0], stones[2], 7, 27))  # Test with Hailstone A and B

None


In [ ]:
total = 0
low,hi = 200000000000000,400000000000000
for i in range(len(stones)):
  for j in range(i,len(stones)):
    res = will_paths_intersect(stones[i], stones[j], low, hi)
    if res:
      total += 1
total

12015

In [ ]:
def will_paths_intersect_3d_single_t(stone1, stone2):
    # Extract coordinates and deltas from the stones
    (x1, y1, z1), (dx1, dy1, dz1) = stone1
    (x2, y2, z2), (dx2, dy2, dz2) = stone2

    # Check for parallel movement in all axes to avoid division by zero
    if dx1 == dx2 and dy1 == dy2 and dz1 == dz2:
        return None

    # Solving for t where paths intersect: x1 + dx1*t = x2 + dx2*t, y1 + dy1*t = y2 + dy2*t, z1 + dz1*t = z2 + dz2*t
    try:
        # Calculate t for each axis
        t_x = (x2 - x1) / (dx1 - dx2) if dx1 != dx2 else None
        t_y = (y2 - y1) / (dy1 - dy2) if dy1 != dy2 else None
        t_z = (z2 - z1) / (dz1 - dz2) if dz1 != dz2 else None

        # Check if the calculated t values are the same for all non-None axes
        t_values = [t for t in [t_x, t_y, t_z] if t is not None]
        if len(set(t_values)) == 1:
            t = t_values[0]
            # Check if t is positive
            if t >= 0:
                intersect_x = x1 + dx1 * t
                intersect_y = y1 + dy1 * t
                intersect_z = z1 + dz1 * t
                return t, intersect_x, intersect_y, intersect_z
    except ZeroDivisionError:
        pass

    return None

will_paths_intersect_3d_single_t(stones[0], ((24, 13, 10), (-3, 1, 2)))

In [ ]:
# pip install z3-solver

In [ ]:
from z3 import Real, Solver, And

def transpose_hailstones(hailstones):
    # Choose the first hailstone's position as the reference
    ref_pos = hailstones[0][0]

    # Transpose all hailstones to the origin based on the reference position
    transposed_hailstones = [((hx - ref_pos[0], hy - ref_pos[1], hz - ref_pos[2]), velocity)
                             for ((hx, hy, hz), velocity) in hailstones]
    return transposed_hailstones, ref_pos

def solve_transposed_hailstones(hailstones):
    solver = Solver()

    # Variables for rock's initial position and velocity (as real numbers)
    rx, ry, rz = Real('rx'), Real('ry'), Real('rz')
    rvx, rvy, rvz = Real('rvx'), Real('rvy'), Real('rvz')

    for i, ((hx, hy, hz), (hvx, hvy, hvz)) in enumerate(hailstones):
        t = Real(f't_{i}')

        solver.add(rx + rvx * t == hx + hvx * t)
        solver.add(ry + rvy * t == hy + hvy * t)
        solver.add(rz + rvz * t == hz + hvz * t)
        solver.add(t > 0)

    print(solver.check())
    model = solver.model()
    return (model[rx].as_decimal(10), model[ry].as_decimal(10), model[rz].as_decimal(10)), \
            (model[rvx].as_decimal(10), model[rvy].as_decimal(10), model[rvz].as_decimal(10))

def solve_hailstone_collision(hailstones):
    transposed_hailstones, ref_pos = transpose_hailstones(hailstones)
    result = solve_transposed_hailstones(transposed_hailstones)

    if result:
        position, velocity = result
        # Transpose the position back to the original coordinates
        transposed_position = (float(position[0]) + ref_pos[0],
                               float(position[1]) + ref_pos[1],
                               float(position[2]) + ref_pos[2])
        return transposed_position, velocity
    else:
        return None

result = solve_hailstone_collision(stones)

if result:
    position, velocity = result
    print("Rock Position:", position, "Rock Velocity:", velocity)
else:
    print("No solution found")

sat
Rock Position: (472612107765508.0, 270148844447628.0, 273604689965980.0) Rock Velocity: ('-333', '-5', '15')


In [ ]:
sum(position)

1016365642179116.0